In [ ]:
%pip install dspy-ai
%pip install ipywidgets
%pip install IPython
%pip install requests
%pip install markdownify
%pip install openai

In [3]:
import dspy
import os
import dotenv
import pydantic
import json
from IPython.display import Markdown, display

dotenv.load_dotenv() #load via .env in this folder (.env is in .gitignore)
#os.environ['OPENAI_API_KEY'] = 'sk-YOUR_OPENAI_API_KEY' #or set directly here, just remember not to commit to GitHub
assert 'OPENAI_API_KEY' in os.environ

first do without message history or vision

In [ ]:
llm = dspy.OpenAI(model='gpt-4o', temperature=0.1, max_tokens=4096) #later need to add vision
class Chat(dspy.Signature):
    """You are a helpful assistant that is always improving based on user feedback. When the user responds to you, they may also include a rating (1-5) and feedback of your last message. Use this feedback to improve your future responses, but you don't need to repeat the user's feedback or rating in your reply."""

    query = dspy.InputField(desc="This is the user's current query.")
    rating = dspy.InputField(desc="This is the user's rating of your last response.")
    feedback = dspy.InputField(desc="This is the user's feedback on your last response.")
    response = dspy.OutputField(desc="This is your response to the user's query. Only output your response in Markdown, do not include any additional text.") #sometimes the AI responds with the output description, so i often leave this blank


class ChatModule(dspy.Module):  # let's define a new module
    def __init__(self):
        super().__init__()
        self.n = 1 #don't do more than 1 right now...
        self.signature = Chat
        self.predictor = dspy.Predict(self.signature, n=self.n) #this can easily be subbed for some other predictor
    
    def forward(self, query, rating=None, feedback=None, num_generations = 1):#don't do more than 1 generation right now...
        self.predictor = dspy.Predict(self.signature, n=num_generations)
        result = self.predictor(query=query, rating=rating, feedback=feedback, n=num_generations)
        return dspy.Prediction(response=[str(completion.response) for completion in result.completions])

In [ ]:
images = client.images.generate(
  model="dall-e-3",
  prompt=image_prompt,
  n=1,
  size="1792x1024"
)